<a href="https://colab.research.google.com/github/OliverGriffiths287/OliverGriffiths287.github.io/blob/main/python_code/ticket_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import requests
import re
import json
import pandas as pd

In [55]:
from mmap import PAGESIZE
url = "https://www.telegraph.co.uk/sport/football/competitions/premier-league/11000182/Premier-League-2014-15-tickets-guide-club-by-club.html?msockid=38f1f384d684602a1958e001d7426184&ICID=continue_without_subscribing_reg_first"

req = requests.get(url)
page_src = req.text

start_index = page_src.index('<p><b><a href="http://www.telegraph.co.uk/sport/football/teams/arsenal/')
page_src = page_src[start_index:]

page_src = page_src.split('<div class="tpl-article__end-meta">')[0]


page_src

'<p><b><a href="http://www.telegraph.co.uk/sport/football/teams/arsenal/">Arsenal</a></b></p>\n       </div>\n      </div>\n      <div class="articleBodyText section">\n       <div class="component article-body-text" data-test="article-body-text">\n        <p>• Cheapest ticket: £27</p>\n       </div>\n      </div>\n      <div class="articleBodyText section">\n       <div class="component article-body-text" data-test="article-body-text">\n        <p>• Most Expensive: £129.50</p>\n       </div>\n      </div>\n      <div class="articleBodyText section">\n       <div class="component article-body-text" data-test="article-body-text">\n        <p>Dependant on Category of the game (A, B, or C). Cheapest category is C. Cheapest seats are behind the goal or near the corner flags on the lower tiers of the stadium.</p>\n       </div>\n      </div>\n      <div class="articleBodyText section">\n       <div class="component article-body-text" data-test="article-body-text">\n        <p><b>Ticket Offi

In [56]:
from os import name
# Find all team entries using regex
team_entries = re.findall(r'/">(.*?)</a></b></p>', page_src)

dfteams = pd.DataFrame(team_entries)
dfteams.rename(columns={0: 'team'}, inplace=True)
dfteams = dfteams[dfteams['team'] != 'Manchester City']
dfteams = dfteams[dfteams['team'] != 'Crystal Palace']
dfteams = dfteams[dfteams['team'] != 'Southampton']
dfteams.reset_index(drop=True, inplace=True)
dfteams

,team
0,Arsenal
1,Aston Villa
2,Burnley
3,Chelsea
4,Everton
5,Hull City
6,Leicester City
7,Liverpool
8,Manchester United
9,Newcastle United


In [57]:
from os import name
# Find all team entries using regex
cheap_tickets_entries = re.findall(r'Cheapest ticket: £(.*?)</p>\n       </div>\n      </div>\n', page_src)
cheap_entries = re.findall(r'Cheapest: £(.*?)</p>\n       </div>\n      </div>\n', page_src)

cheap_entries = cheap_tickets_entries + cheap_entries

dfcheap = pd.DataFrame(cheap_entries)
dfcheap.rename(columns={0: 'cheapest'}, inplace=True)
dfcheap['cheapest'] = dfcheap['cheapest'].str[:2]
dfcheap

,cheapest
0,27
1,20
2,35
3,36
4,33
5,16
6,26
7,37
8,31
9,24


In [25]:
dfcombine = pd.concat([dfteams, dfcheap], axis=1)
dfcombine

dfcombine.to_csv('cheap_tickets.csv', index=False)

In [61]:
from mmap import PAGESIZE
import requests
from bs4 import BeautifulSoup
url = "https://www.givemesport.com/every-premier-league-club-ranked-by-their-cheapest-match-day-ticket-price/"
req = requests.get(url)
page_src2 = req.text

start_index = page_src2.index('<div class="table-container">')
page_src2 = page_src2[start_index:]

page_src2 = page_src2.split('<!-- No repeatable ad for zone: character count repeatable. --><!-- Repeatable debug data:')[0]


page_src2

'<div class="table-container">\n        <table border="1" cellpadding="1" cellspacing="1" style="" align="" summary="">\n                    \n\n<tbody >\n    \n\n<tr  >\n            \n\n    <td >\n\n\n        \n<p>Everton</p>    \n\n    </td>\n\n            \n\n    <td >\n\n\n        \n<p>&pound;55</p>    \n\n    </td>\n\n    </tr>\n    \n\n<tr  >\n            \n\n    <td >\n\n\n        \n<p>Newcastle United</p>    \n\n    </td>\n\n            \n\n    <td >\n\n\n        \n<p>&pound;44</p>    \n\n    </td>\n\n    </tr>\n    \n\n<tr  >\n            \n\n    <td >\n\n\n        \n<p>Manchester City</p>    \n\n    </td>\n\n            \n\n    <td >\n\n\n        \n<p>&pound;44</p>    \n\n    </td>\n\n    </tr>\n    \n\n<tr  >\n            \n\n    <td >\n\n\n        \n<p>Crystal Palace</p>    \n\n    </td>\n\n            \n\n    <td >\n\n\n        \n<p>&pound;42</p>    \n\n    </td>\n\n    </tr>\n    \n\n<tr  >\n            \n\n    <td >\n\n\n        \n<p>Aston Villa</p>    \n\n    </td>\n\n 

In [63]:
from os import name
# Find all team entries using regex
team_entries = re.findall(r'<td >\n\n\n        \n<p>(.*?)</p>    \n\n    </td>', page_src2)

df_teams2 = pd.DataFrame({'odd_values': team_entries[::2]})
df_teams2.rename(columns={'odd_values': 'team'}, inplace=True)
df_teams2.replace('&amp;', '')

df_tickets = pd.DataFrame({'even_values': team_entries[1::2]})
df_tickets.rename(columns={'even_values': 'cheap'}, inplace=True)
df_tickets['cheap24'] = df_tickets['cheap'].str.split(';').str[1]
df_tickets = df_tickets.drop(columns=['cheap'])
df_tickets

,cheap24
0,55
1,44
2,44
3,42
4,40.50
5,38
6,37
7,36
8,35
9,35


In [70]:
dfcheap_tickets24 = pd.concat([df_teams2, df_tickets], axis=1)
dfcheap_tickets24.to_csv('cheap_tickets24.csv', index=False)
